In [1]:
# Import required packages
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
import logging
from azureml.core.model import Model
from azureml.exceptions import WebserviceException
import os, shutil

## Setting up Key Vault Values - Do not keep these around - run 1 time only

In [2]:
# import azureml.core
# import os, shutil
# from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
# from azureml.core.authentication import ServicePrincipalAuthentication

# os.environ.setdefault('tenantId', 'XXXX')
# os.environ.setdefault('servicePrincipalId', 'XXXX')
# os.environ.setdefault('servicePrincipalPassword', 'XXXXXX')
# os.environ.setdefault('wsName', 'XXX')
# os.environ.setdefault('subscriptionId', 'XXXXX')
# os.environ.setdefault('resourceGroup', 'XXXXX')


# environment_variables = ['tenantId', 'servicePrincipalId', 'servicePrincipalPassword', 'wsName', 
#                          'subscriptionId', 'resourceGroup']

# envs = {}

# for x in environment_variables:
#     print(x, "=", os.environ.get(x))
#     envs[x] = os.environ.get(x)


# sp = ServicePrincipalAuthentication(tenant_id=envs['tenantId'], # tenantID
#                                     service_principal_id=envs['servicePrincipalId'], # clientId
#                                     service_principal_password=envs['servicePrincipalPassword']) # clientSecret

# ws = Workspace.get(name=envs['wsName'],
#                    auth=sp,
#                    subscription_id=envs['subscriptionId'],
#                    resource_group=envs['resourceGroup'])
# ws.get_details()
# keyvault = ws.get_default_keyvault()

# for x in environment_variables:
#     print(x, "=", os.environ.get(x))
#     keyvault.set_secret(name = x, value = envs[x])

In [3]:
# def get_environment_variables():
#     global envs
#     global run_by_notebook 
#     run_by_notebook = False
#     environment_variables = ['tenantId', 'servicePrincipalId', 'servicePrincipalPassword', 'wsName', 
#                          'subscriptionId', 'resourceGroup']
#     envs = {}
#     for x in environment_variables:
#         if os.environ.get(x) == None:
#             #get the values from keyvault
#             run_by_notebook = True
#             print('retrieve from key vault, value is None: ' + x)
#             ws = Workspace.from_config()
#             keyvault = ws.get_default_keyvault()
#             kv_results = keyvault.get_secrets(environment_variables)
#             envs = kv_results
#             for x in envs:
#                 os.environ.setdefault(x, envs[x])
#             exit
#         else:
#             envs[x] = os.environ.get(x)
#     return run_by_notebook



# get_environment_variables()
# sp = ServicePrincipalAuthentication(tenant_id=envs['tenantId'], # tenantID
#                                     service_principal_id=envs['servicePrincipalId'], # clientId
#                                     service_principal_password=envs['servicePrincipalPassword']) # clientSecret
# ws = Workspace.get(name=envs['wsName'],
#                        auth=sp,
#                        subscription_id=envs['subscriptionId'],
#                        resource_group=envs['resourceGroup'])
# ws.get_details()

# print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
# print('Run By Notebook:' + str(run_by_notebook))
# # Get the default datastore
# default_ds = ws.get_default_datastore()

In [4]:
ws = Workspace.from_config()
default_ds = ws.get_default_datastore()
run_by_notebook = True

In [5]:
cluster_name = 'aml-cluster001'
model_name_to_register = 'text_classification_001'
experiment_folder = 'text_classificatin_example'
conda_yml_file = 'textclassification_env.yml'
registered_env_name = "text-classificiation-env"

In [6]:

#Select AML Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [7]:
try:
    initial_model = Model(ws, model_name_to_register)
    inital_model_version = initial_model.version
except WebserviceException :
    inital_model_version = 0
print('inital_model_version = ' + str(inital_model_version))

inital_model_version = 4


In [8]:
import os
import shutil
# Create a folder for the pipeline step files
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

run_path = './run_outputs'
try:
    shutil.rmtree(run_path)
except:
    print('continue run_outputs directory does not exits')

text_classificatin_example
continue run_outputs directory does not exits


In [9]:
env = Environment.from_conda_specification(registered_env_name, conda_yml_file)
env.register(workspace=ws)

{
    "assetId": "azureml://locations/eastus2/workspaces/9634b616-29cb-4345-ae22-4be4e4bfe009/environments/text-classificiation-env/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221101.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "text-classif

In [10]:
try:
    registered_env = Environment.get(ws, registered_env_name)
    pipeline_run_config = RunConfiguration()
    
    # Use the compute you created above. 
    pipeline_run_config.target = pipeline_cluster

    # Assign the environment to the run configuration
    pipeline_run_config.environment = registered_env
    print ("Run configuration created.")
except Exception as e: 
    print(e)

Run configuration created.


## Define Output datasets

The OutputFileDatasetConfig object is a special kind of data reference that is used for interim storage locations that can be passed between pipeline steps, so you'll create one and use at as the output for the first step and the input for the second step. Note that you need to pass it as a script argument so your code can access the datastore location referenced by the data reference.

Note, in all cases we specify the datastore that should hold the datasets and whether they should be registered following step completion or not. This can optionally be disabled by removing the register_on_complete() call.

These can be viewed in the Datasets tab directly in the AML Portal

In [ ]:
model_name_to_register

In [ ]:
training_data  = OutputFileDatasetConfig(name='training_data', destination=(default_ds, model_name_to_register + '_training_data/{run-id}')).read_delimited_files().register_on_complete(name= model_name_to_register + '_training_data')
testing_data   = OutputFileDatasetConfig(name='testing_data',  destination=(default_ds, model_name_to_register +  '_testing_data/{run-id}')).read_delimited_files().register_on_complete(name= model_name_to_register + '_testing_data')

model_file     = PipelineData(name='model_file', datastore=default_ds)

model_name         = PipelineParameter("model_name", default_value= model_name_to_register)
model_desc         = PipelineParameter("model_desc", default_value=model_name_to_register + ' description')
raw_file_location  = PipelineParameter(name="raw_file_location", default_value='spam-data/spamformodel.csv')

In [ ]:
%%writefile $experiment_folder/classifier_training.py

import argparse
from azureml.core import Run, Workspace, Datastore, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
import joblib
import os
import shutil


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


def summarize_classification(y_test, y_pred, run):
    acc = accuracy_score(y_test, y_pred, normalize=True) #how many predictions correct %
    num_acc = accuracy_score(y_test, y_pred, normalize = False)
    prec = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    
    run.log('total count', len(y_test))
    run.log('acc count', num_acc)
    run.log('Accuracy', acc)
    run.log('prec', prec)
    run.log('recall', recall)
    
    run.parent.log('acc count', num_acc)
    run.parent.log('Accuracy', acc)
    run.parent.log('prec', prec)
    run.parent.log('recall', recall)


    print('accuracy count:', num_acc)
    print('accuracy score:', acc)
    print('precision:', prec)
    print('recall:', recall)
    


def getRuntimeArgs():
    # Get script arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('--raw_file_location', dest='raw_file_location', required=True)
    parser.add_argument('--training_data', dest='training_data', required=True)
    parser.add_argument('--testing_data', dest='testing_data', required=True)
    parser.add_argument('--model_file', dest='model_file', required=True)
    parser.add_argument('--model_name', dest='model_name', required=True)
    args = parser.parse_args()

    return args

def model_train(ds_df, run, training_data, testing_data):
    
    clf = Pipeline([
                            ('count_vectorizer', CountVectorizer()),
                            ('classifier', LogisticRegression(solver='lbfgs', max_iter=10000))
                        ])
    #output of convectorizer, feed to classifier
    train, test = train_test_split(ds_df, test_size=0.2, random_state=0)

    x_train = train['text']
    y_train = train['labels']
    
    x_test = test['text']
    y_test = test['labels']
    
    
    print(train.head())


    model = clf.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    print('*************************')
    print('model predictions:')
    print(y_pred)
    summarize_classification(y_test, y_pred, run)
    
    os.makedirs(training_data, exist_ok=True)
    os.makedirs(testing_data, exist_ok=True)

    test['results'] = y_pred
    print('training_data = ' + training_data)
    print('testing_data = ' + testing_data)
    train.to_csv(os.path.join(training_data, 'training_data.csv'), index=False )
    test.to_csv(os.path.join(testing_data, 'testing_data.csv'), index=False)

    return model



def main():
    args = getRuntimeArgs()
    
    raw_file_location = args.raw_file_location
    training_data = args.training_data
    testing_data = args.testing_data
    model_file = args.model_file
    model_name = args.model_name
    
    print("training_data =" + training_data)
    print('testing_data =' + testing_data)
    # Get the experiment run context
    run = Run.get_context()
    ws = run.experiment.workspace
    print(ws)
    ds = ws.get_default_datastore()
    
    
    print("Loading Data...")
    dataset = Dataset.Tabular.from_delimited_files(path = [(ds, raw_file_location)])
    data = dataset.to_pandas_dataframe()
    
    
    print(data.columns)
    lr = model_train(data, run, training_data,  testing_data)
    
    os.makedirs('./outputs', exist_ok=True)
    model_file_name = model_name  + '.pkl'
    file_name = './outputs/' +model_file_name
    
    print("Joblib Version : ", joblib.__version__)
    
    joblib.dump(value=lr, filename=file_name)

    #copy to pass model to next step as the model file
    os.makedirs(model_file, exist_ok=True)
    shutil.copyfile(file_name, os.path.join(model_file, model_file_name))

    run.complete()

 

if __name__ == "__main__":
    main()

## Train Model Python Script Step

In [ ]:
train_model_step = PythonScriptStep(
    name='Get Data and Create Model',
    script_name='classifier_training.py',
    arguments =['--raw_file_location', raw_file_location,
                '--training_data', training_data,
                '--testing_data', testing_data,
                '--model_file', model_file,
                '--model_name', model_name
               ],
    inputs=[],
    outputs=[model_file, training_data, testing_data],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

## Evaluate Model

In [ ]:
%%writefile $experiment_folder/eval.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.core.model import Model
from azureml.data.datapath import DataPath

import joblib
import os
import argparse
import shutil
import pandas as pd
import shutil


from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice


parser = argparse.ArgumentParser("Evaluate model and register if more performant")

parser.add_argument('--model_name', dest='model_name', required=True)
parser.add_argument('--model_file', dest = 'model_file',  required=True)
parser.add_argument('--model_desc', dest = 'model_desc',  required=True)
parser.add_argument('--deploy_file', dest='deploy_file', required=True)


args, _ = parser.parse_known_args()
model_name = args.model_name
model_file = args.model_file
model_desc = args.model_desc

deploy_file = args.deploy_file

#Get current run
run = Run.get_context()

#Get associated AML workspace
ws = run.experiment.workspace

#Get default datastore
ds = ws.get_default_datastore()

#Get metrics associated with current parent run
metrics = run.get_metrics()

print('current run metrics')
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')


print('parent run metrics')
#Get metrics associated with current parent run
metrics = run.parent.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')


current_model_acc_count = float(metrics['acc count'])
current_model_acc = float(metrics['Accuracy'])
current_model_prec = float(metrics['prec'])
current_model_recall = float(metrics['recall'])
    

# Get current model from workspace

model_description = model_desc
model_list = Model.list(ws, name=model_name, latest=True)
first_registration = len(model_list)==0

updated_tags = {'Accuracy': current_model_acc, 'prec': current_model_prec, 'recall': current_model_recall}


print('updated tags')
print(updated_tags)


#upload model to the outputs directory
relative_model_path = 'outputs'
run.upload_folder(name=relative_model_path, path=model_file)

model_file_name = model_name  + '.pkl'

###################
test_model = joblib.load(model_file + '/' + model_file_name)
test_dataset = run.input_datasets['testing_data']
df_test = test_dataset.to_pandas_dataframe()
X = df_test['text']
# Make predictions with new dataframe
predictions = test_model.predict(X)
print('predictions')
print(predictions)
#########################





#If no model exists register the current model
if first_registration:
    print('First model registration.')
    model_reg = run.register_model(model_path='outputs/' + model_file_name, model_name=model_name,
                   tags=updated_tags,
                   properties=updated_tags)
else:
    #If a model has been registered previously, check to see if current model 
    #performs better. If so, register it.
    print(dir(model_list[0]))
    if float(model_list[0].tags['prec']) < current_model_prec:
        print('New model performs better than existing model. Register it.')

        model_reg = run.register_model(model_path='outputs/' + model_file_name, model_name=model_name,
                   tags=updated_tags,
                   properties=updated_tags)
        
        # Output accuracy to file
        with open(deploy_file, 'w+') as f:
            f.write(('deploy_model'))
    
    else:
        print('New model does not perform better than existing model. Cancel run.')
        
        with open(deploy_file, 'w+') as f:
            f.write(('do not deploy model'))
            
        run.complete()

In [ ]:
deploy_file = PipelineData(name='deploy_file', datastore=default_ds)

evaluate_and_register_step = PythonScriptStep(
    name='Evaluate and Register Model',
    script_name='eval.py',
    arguments=[
        '--model_name', model_name,
        '--model_file', model_file,
        '--model_desc', model_desc,
        '--deploy_file', deploy_file,       
    ],
    inputs=[model_file.as_input('model_file'),
            training_data.as_input(name='training_data'),
            testing_data.as_input(name='testing_data')
           ],
    outputs=[ deploy_file],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [ ]:
%%writefile $experiment_folder/deploy.py

import argparse
from azureml.core import Workspace, Environment
from azureml.core.model import Model
from azureml.core.run import Run
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice
from azureml.exceptions import WebserviceException

parser = argparse.ArgumentParser(description='Deploy arg parser')
parser.add_argument('--scoring_file_output', type=str, help='File storing the scoring url')
parser.add_argument('--deploy_file', type=str, help='File storing if model should be deployed')
parser.add_argument('--environment_name', type=str,help='Environment name')
parser.add_argument('--service_name', type=str,help='service name')
parser.add_argument('--model_name', type=str,help='model name')



args = parser.parse_args()
scoring_url_file = args.scoring_file_output
deploy_file      = args.deploy_file
environment_name = args.environment_name
service_name     = args.service_name
model_name       = args.model_name


run = Run.get_context()

#Get associated AML workspace
ws = run.experiment.workspace

model = Model(ws, model_name)
env = Environment.get(ws, environment_name)
inference_config = InferenceConfig(entry_script='score.py', environment=env)

#test_dataset = run.input_datasets['testing_data']

print(deploy_file)
with open(deploy_file, 'r+') as f:
    result = f.read()

print(result)
print(type(result))

if result == 'do not deploy model':
    print('do not update the deployed model')
    run.complete()

else:
    print('continue with deploying model...')        
    # Deploy model
    aci_config = AciWebservice.deploy_configuration(
                cpu_cores = 1, 
                memory_gb = 2, 
                tags = {'model': 'diabetes remote training'},
                auth_enabled=True,
                enable_app_insights=True)

    try:
        service = Webservice(ws, name=service_name)
        if service:
            service.delete()
    except WebserviceException as e:
             print()

    service = Model.deploy(ws, service_name, [model], inference_config, aci_config, overwrite=True)
    service.wait_for_deployment(True)


    # Output scoring url
    print(service.scoring_uri)
    with open(scoring_url_file, 'w+') as f:
        f.write(service.scoring_uri)

In [ ]:
%%writefile $experiment_folder/score.py

import json
import joblib
import numpy as np
from azureml.core.model import Model
import time
import os
import pandas as pd


#version 2
# Called when the service is loaded
def init():
    global model
    #Print statement for appinsights custom traces:
    print ("model initialized" + time.strftime("%H:%M:%S"))
    # Get the path to the deployed model file and load it
    path = os.path.join(Model.get_model_path('text_classification_001'))
    
    print(path)
    model = joblib.load(path)




# Called when a request is received
def run(raw_data):
    try:
        # Get the input data as a numpy array
        #data = np.array(json.loads(raw_data)['data'])
        # Get a prediction from the model
        
        json_data = json.loads(raw_data)
        predictions = model.predict(json_data['data'])
        print ("Prediction created" + time.strftime("%H:%M:%S"))
        print(predictions)
        # Get the corresponding classname for each prediction (0 or 1)
        #classnames = ['not-diabetic', 'diabetic']
        predicted_classes = []
        for prediction in predictions:
            #val = int(prediction)
            predicted_classes.append(prediction)
        # Return the predictions as JSON
        
         # Log the input and output data to appinsights:
        info = {
            "input": raw_data,
            "output": predicted_classes
            }
        print(json.dumps(info))
        

        return json.dumps(predicted_classes)
    except Exception as e:
        error = str(e)
        print (error + time.strftime("%H:%M:%S"))
        return error

In [ ]:
scoring_file = PipelineData(name='scoring_file', datastore=default_ds)

aci_service_name = 'text-classification'

env_name = PipelineParameter(name='environment_name', default_value=registered_env_name)
service_name = PipelineParameter(name='service_name', default_value=aci_service_name)
scoring_file = PipelineData(name='scoring_file', datastore=default_ds)

########################################################

deploy_test = PythonScriptStep(
    name='Deploy to ACI',
    script_name='deploy.py',
    arguments=[
        '--scoring_file_output', scoring_file,
        '--deploy_file', deploy_file,
        '--environment_name', env_name,
        '--service_name', service_name,
        '--model_name', model_name
        
    ],
    inputs=[
        deploy_file.as_input('deploy_file'),
            
    ],
    outputs=[scoring_file],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [ ]:
## Create Pipeline Steps
pipeline = Pipeline(workspace=ws, steps=[train_model_step, evaluate_and_register_step, deploy_test])
if run_by_notebook:
    experiment = Experiment(ws, 'AML_Manual_PipelineTraining')
else:
    experiment = Experiment(ws, 'AML_AutoDevOps_PipelineTraining')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

In [ ]:
import json

try:
    final_model = Model(ws, model_name_to_register)
    final_model_version = final_model.version
except WebserviceException :
    final_model_version = 0
    
print('inital_model_version = ' + str(inital_model_version))
print('final_model_version= ' + str(final_model_version))

status = run.get_status()
run_details = run.get_details()

print((run_details))
print(run_details['runId'])

In [ ]:
if final_model_version > 0 and (inital_model_version != final_model_version):
    deploy = 'deploy'
    model_details = {
        "name" : final_model.name,
        "version": final_model.version,
        "properties": final_model.properties,
        "nextstep": "deploy"
    }
    print(model_details)
else:
    deploy = 'no'
    print('do not deploy')

In [ ]:
for x in final_model.properties:
    print(x)
    print(final_model.properties[x])

print (final_model.properties)

In [ ]:
import json
import shutil
import os

outputfolder = 'run_outputs'
os.makedirs(outputfolder, exist_ok=True)

if (final_model_version != inital_model_version):
    print('new model registered')
    with open(os.path.join(outputfolder, 'deploy_details.json'), "w+") as f:
        f.write(str(model_details))
    model_name = model_name_to_register
    model_list = Model.list(ws, name=model_name_to_register, latest=True)
    model_path = model_list[0].download(exist_ok=True)
    model_file_name = model_name_to_register + '.pkl'
    shutil.copyfile(model_file_name,  os.path.join(outputfolder,model_file_name))
    
    #create model.yml file.
    with open(os.path.join(outputfolder, 'model.yml'), "w+") as f:
        f.write('$schema: https://azuremlschemas.azureedge.net/latest/model.schema.json \n')
        f.write('name: ' + model_details['name'] + '\n')
        f.write('path: ' + model_name_to_register + '.pkl \n')
        f.write('description: Model created from local file. \n')
        if len(final_model.properties) > 0:
            f.write('properties: ')
            f.write(json.dumps(final_model.properties))
            f.write('\n')
            f.write('tags: ')
            f.write(json.dumps(final_model.properties))
            
    
with open(os.path.join(outputfolder, 'run_details.json'), "w+") as f:
    print(run_details)
    f.write(str(run_details))

with open(os.path.join(outputfolder, "run_number.json"), "w+") as f:
    f.write(run_details['runId'])
    
with open(os.path.join(outputfolder, "deploy.txt"), "w+") as f:
    f.write(deploy)

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

def published_pipeline_to_pipeline_endpoint(
    workspace,
    published_pipeline,
    pipeline_endpoint_name,
    pipeline_endpoint_description="Endpoint to Email Classification Training pipeline",
):
    try:
        pipeline_endpoint = PipelineEndpoint.get(
            workspace=workspace, name=pipeline_endpoint_name
        )
        print("using existing PipelineEndpoint...")
        pipeline_endpoint.add_default(published_pipeline)
    except Exception as ex:
        print(ex)
        # create PipelineEndpoint if it doesn't exist
        print("PipelineEndpoint does not exist, creating one for you...")
        pipeline_endpoint = PipelineEndpoint.publish(
            workspace=workspace,
            name=pipeline_endpoint_name,
            pipeline=published_pipeline,
            description=pipeline_endpoint_description
        )

if deploy == 'deploy':
    print('deploy Email Classification Training Pipeline')
    pipeline_endpoint_name = 'Email Classification Training Pipeline'
    pipeline_endpoint_description = 'Endpoint to Email Classification Training pipeline'

    published_pipeline = pipeline.publish(name=pipeline_endpoint_name,
                                         description=pipeline_endpoint_description,
                                         continue_on_step_failure=False)

    published_pipeline_to_pipeline_endpoint(
        workspace=ws,
        published_pipeline=published_pipeline,
        pipeline_endpoint_name=pipeline_endpoint_name,
        pipeline_endpoint_description=pipeline_endpoint_description
    )
else:
    print('do not publish pipeline')

## Testing out endpoint

In [ ]:
import json
import requests
from azureml.core.webservice import Webservice


#get service endpoint
service = Webservice(workspace=ws, name='text-classification')
print(service.state)
url = service.scoring_uri
print(url)

primary, secondary = service.get_keys()
print(primary)
api_key = primary
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}



def MakePrediction():
    endpoint_url = url
    x_new = ['this is data']
    input_json = json.dumps({"data": x_new})
    print(input_json)
    body = input_json
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())


results = MakePrediction()
print(results)


